# Basic Usage

You can invoke `main.py` as such
`python main.py -q {query} -i {input xlsx file} -o {output xlsx file}`

Or you can use programmatically as below

Make sure `OPENAI_API_KEY` is set in your environment.

In [2]:
from highlighter import FinancialDataHighlighter

input_book = '../data/sample_input.xlsx'
output_book = '../data/sample_input_revenue.xlsx'
query = "Revenue"

analyst = FinancialDataHighlighter()
analyst.analyze_workbook(query, input_book, output_book)

# Evaluation

Below is the scaffolding for an evaluation system that would be used to lead further development of the system

In [10]:
# For eval:
from dataclasses import dataclass
import openpyxl

@dataclass
class EvalExample:
    query: str # e.g. 'Revenue'
    workbook_file: str # e.g. '../data/sample_input.xlsx'
    golden_label_cells: dict[str, list[tuple[int, int]]] # Cells that *should* get highlighted in each worksheet {"NOW-US, IS FY'23": [(14,2), (14,3), (14,4)]} 

examples = [
    EvalExample('Revenue', '../data/sample_input.xlsx', {
        "NOW-US, IS FY'23": [(14,2), (14,3), (14,4)]
    })
]

analyst = FinancialDataHighlighter()

eval_results = {}
for example in examples:
    wb = openpyxl.load_workbook(example.workbook_file)
    predicted_cells = analyst._get_worksheet_cells(example.query, wb)
    true_positives, false_positives, false_negatives = 0, 0, 0

    for sheetname in wb.sheetnames:
        predicted = predicted_cells.get(sheetname, [])
        true_positives += len([val for val in predicted if val in example.golden_label_cells.get(sheetname, [])])
        false_positives += len([val for val in predicted if val not in example.golden_label_cells.get(sheetname, [])])
        false_negatives += len([val for val in example.golden_label_cells.get(sheetname, []) if val not in predicted])

    eval_results[f"{example.workbook_file}/{example.query}"] = (true_positives, false_positives, false_negatives)

print("Eval example: (correct labels, false positives, false negatives)")
print(eval_results)

Eval example: (correct labels, false positives, false negatives)
{'../data/sample_input.xlsx/Revenue': (2, 7, 1)}


# Future ideas
* add in context examples
* Fix observed issues:
    * column index consistency seems to be off, examples might help, different labeling strategies might help
* Test with different financial statement document formats beyond the example data
* Maybe do a second pass asking the LLM if the extracted cells are actually relevant
* If first pass does not highlight any data, do a 2 pass approach where you generate potential synonyms or relevant data types on the first pass and try to highlight those. (depends on precision/recall requirements of the product in question) 
* try a multi model approach, sending easier queries to gpt-3.5 (depends if model call optimizations are needed in a production scenario)